In [79]:
import pandas as pd
import numpy as np

import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity

In [131]:
book_df=pd.read_csv('/kaggle/input/book-recommendation-dataset/Books.csv')
rating_df=pd.read_csv('/kaggle/input/book-recommendation-dataset/Ratings.csv')
user_df=pd.read_csv('/kaggle/input/book-recommendation-dataset/Users.csv')

/tmp/ipykernel_32/1373560247.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_df=pd.read_csv('/kaggle/input/book-recommendation-dataset/Books.csv')


In [132]:
book_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [49]:
rating_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [50]:
user_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [51]:
print(book_df.shape)
print(rating_df.shape)
print(user_df.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


In [130]:
book_df.describe()

,ISBN,Book-Title,Year-Of-Publication,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271360,271360,271360,271357
unique,271360,242135,202,271044,271044,271041
top,0195153448,Selected Poems,2002,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/225307649X.0...
freq,1,27,13903,2,2,2


In [53]:
# 2 missing value in book records.
#we can either drop these records or fill na with the mode.
book_df.isnull().sum()
book_df.drop(columns=['Book-Author','Publisher'], inplace=True)

In [54]:
rating_df.describe()

,User-ID,Book-Rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [55]:
# No missing value
rating_df.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [56]:
# Total user records: 278858, 40% of age data is missing in User Records

user_df.isnull().sum()
# print(110762/278858)

User-ID          0
Location         0
Age         110762
dtype: int64

## Popularity Based Recommendation System

In [57]:
books=book_df[['ISBN', 'Book-Title']]
ratings=rating_df[['ISBN', 'Book-Rating']]
books_rating_df=books.merge(ratings, on='ISBN')
books_rating_df.head(2)

,ISBN,Book-Title,Book-Rating
0,0195153448,Classical Mythology,0
1,0002005018,Clara Callan,5


In [58]:
agg_books_rating_df=books_rating_df.groupby('Book-Title')['Book-Rating'].agg({'count','mean'}).reset_index()
agg_books_rating_df.rename(columns={'count':'Num-Ratings', 'mean':'Avg-Rating'},inplace=True)
agg_books_rating_df.head()

,Book-Title,Num-Ratings,Avg-Rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.25
1,Always Have Popsicles,1,0.00
2,Apple Magic (The Collector's series),1,0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.00
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.00


In [59]:
filtered_books_rating_df=agg_books_rating_df[agg_books_rating_df['Num-Ratings']>=200]
filtered_books_rating_df=filtered_books_rating_df.sort_values(ascending=False,by=['Avg-Rating'])
filtered_books_rating_df.head(20)

,Book-Title,Num-Ratings,Avg-Rating
80434,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
80422,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
80441,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
80426,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
60582,Ender's Game (Ender Wiggins Saga (Paperback)),249,5.409639
80414,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
191612,The Hobbit : The Enchanting Prelude to The Lor...,281,5.007117
187377,The Fellowship of the Ring (The Lord of the Ri...,368,4.948370
80445,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.895652
211384,"The Two Towers (The Lord of the Rings, Part 2)",260,4.880769


## Colabrative Filtering

In [60]:
book_user_rating_df = rating_df.merge(book_df,on='ISBN')
book_user_rating_df.head()

,User-ID,ISBN,Book-Rating,Book-Title,Year-Of-Publication,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,2002,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,2002,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,2002,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,2002,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,2002,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [61]:
# Filter out users, based on number of ratings they have given
user_rating_count=book_user_rating_df.groupby('User-ID').count()['Book-Rating']>200
filtered_user=user_rating_count[user_rating_count].index

filtered_user_df=book_user_rating_df[book_user_rating_df['User-ID'].isin(filtered_user)]
print('Users with rating count more than 200 : ', len(filtered_user))

Users with rating count more than 200 :  811


In [67]:
book_rating_count=filtered_user_df.groupby('ISBN').count()['Book-Rating']>50
filtered_book=book_rating_count[book_rating_count].index

filtered_df=filtered_user_df[filtered_user_df['ISBN'].isin(filtered_book)]
print('Books with rating count more than 50 : ', len(filtered_book))

Books with rating count more than 50 :  484


In [68]:
filtered_df.shape

(38817, 8)

In [75]:
filtered_df.drop_duplicates()

,User-ID,ISBN,Book-Rating,Book-Title,Year-Of-Publication,Image-URL-S,Image-URL-M,Image-URL-L
184,278418,038550120X,0,A Painted House,2001,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
185,6251,038550120X,0,A Painted House,2001,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
189,11676,038550120X,10,A Painted House,2001,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
190,13273,038550120X,0,A Painted House,2001,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
191,13552,038550120X,0,A Painted House,2001,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
...,...,...,...,...,...,...,...,...
521230,252071,0312971125,0,Crazy for You,2000,http://images.amazon.com/images/P/0312971125.0...,http://images.amazon.com/images/P/0312971125.0...,http://images.amazon.com/images/P/0312971125.0...
521232,258185,0312971125,0,Crazy for You,2000,http://images.amazon.com/images/P/0312971125.0...,http://images.amazon.com/images/P/0312971125.0...,http://images.amazon.com/images/P/0312971125.0...
521233,263877,0312971125,0,Crazy for You,2000,http://images.amazon.com/images/P/0312971125.0...,http://images.amazon.com/images/P/0312971125.0...,http://images.amazon.com/images/P/0312971125.0...
521234,265115,0312971125,10,Crazy for You,2000,http://images.amazon.com/images/P/0312971125.0...,http://images.amazon.com/images/P/0312971125.0...,http://images.amazon.com/images/P/0312971125.0...


In [78]:
book_user_matrix=filtered_df.pivot_table(index='Book-Title',columns='User-ID', values='Book-Rating')
book_user_matrix.fillna(0, inplace=True)
book_user_matrix

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Case of Need,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
similarity_scores = cosine_similarity(book_user_matrix)
similarity_scores.shape

(475, 475)

In [144]:
def book_recomm(book_name):
    book_index=np.where(book_user_matrix.index==book_name)[0][0]
    similar_books=sorted(list(enumerate(similarity_scores[book_index])), key=lambda x:x[1], reverse=True)[1:6]
    
    recomm_list=[]
    for i in similar_books:
        book_details=[]
        book=book_df[book_df['Book-Title'] == book_user_matrix.index[i[0]]]
        book_details.extend(list(book.drop_duplicates('Book-Title')['Book-Title'].values))
        book_details.extend(list(book.drop_duplicates('Book-Title')['Book-Author'].values))
        book_details.extend(list(book.drop_duplicates('Book-Title')['Image-URL-M'].values))
        recomm_list.append(book_details)
        
    return recomm_list
    

In [145]:
book_recomm('Secrets')

[['Kaleidoscope',
  'June Stepansky',
  'http://images.amazon.com/images/P/0884092097.01.MZZZZZZZ.jpg'],
 ['Fine Things',
  'Danielle Steel',
  'http://images.amazon.com/images/P/0440200563.01.MZZZZZZZ.jpg'],
 ['Zoya',
  'Danielle Steel',
  'http://images.amazon.com/images/P/0440203856.01.MZZZZZZZ.jpg'],
 ['Wanderlust',
  'Danielle Steel',
  'http://images.amazon.com/images/P/0440193613.01.MZZZZZZZ.jpg'],
 ['My Gal Sunday',
  'Mary Higgins Clark',
  'http://images.amazon.com/images/P/0684832291.01.MZZZZZZZ.jpg']]